In [25]:
import numpy as np

import altair as alt

import datadotworld as dw

import pandas as pd

import opendatasets as od

In [26]:
# dw.query('https://data.world/mhoangvslev/steam-games-dataset',
# '''
# SELECT
# FROMobject has no attribute 'properties'
# WHERE
# ORDER BY
# LIMIT
# '''
# )


## Find the top 10 Cities with the most amount of game developers

In [27]:
q1 =dw.query('https://data.world/mhoangvslev/steam-games-dataset',
'''
SELECT DISTINCT city AS City, country AS Country, COUNT(developer) as Developer_Count
FROM developper 
GROUP BY city
ORDER BY COUNT(developer) DESC
LIMIT 10
'''
)

In [28]:
(print(q1.dataframe
.to_markdown(index=None)))

| City          | Country     |   Developer_Count |
|:--------------|:------------|------------------:|
| Tokyo         | Japan       |                77 |
| London        | England     |                14 |
| San Francisco | US          |                13 |
| Seoul         | South Korea |                13 |
| Paris         | France      |                12 |
| Osaka         | Japan       |                10 |
| Cambridge     | US          |                10 |
| Los Angeles   | US          |                 9 |
| Austin        | US          |                 9 |
| Chicago       | US          |                 8 |


In [29]:
q1_chart = (alt.Chart(q1.dataframe, title = "Top 10 Cities With The Most Games Developers")
.mark_bar()
.encode(
    color = alt.Color('Country'),
    x = alt.X('Developer_Count'),
    y = alt.Y('City', sort=alt.EncodingSortField(field='Developer_Count', order='ascending', op='sum'))    
)
.properties(
    width = 800,
    height = 400
))

q1_text = q1_chart.mark_text(
    align='left',
    baseline='middle',
    dx = 3
).encode(
    text='Developer_Count'
)

q1_chart+q1_text

alt.LayerChart(...)

In [30]:
q1_chart.save("q1_chart.png")

## Find the Top 5 devleopers with the highest amount of sales (global Sales)

In [31]:
q2 = dw.query('https://data.world/mhoangvslev/steam-games-dataset',
'''
SELECT d.developer AS Developer, sum(v.global_sales)*1000000 AS Global_Sales
FROM vgsales v
JOIN developper d
ON  v.developer = d.developer
GROUP BY d.developer
ORDER BY Global_Sales DESC
LIMIT 5
'''
)

In [32]:
(print(q2.dataframe
.to_markdown(index=None,floatfmt='.0f')))

| Developer         |   Global_Sales |
|:------------------|---------------:|
| Nintendo          |      531710000 |
| Ubisoft           |      132540000 |
| Capcom            |      115710000 |
| Treyarch          |      103160000 |
| Traveller's Tales |       79220000 |


In [33]:
q2_chart = (alt.Chart(q2.dataframe, title = "Top 5 Developers by Global Sales")
.mark_bar()
.encode(
    x= alt.X('Developer', sort='y'),
    y= alt.Y('Global_Sales'),
    color = alt.Color('Global_Sales', legend = None))
.properties(
    width = 800,
    height = 400
))


q2_chart

alt.Chart(...)

In [34]:
q2_chart.save("q2_chart.png")

## Breakdown the amount of games Nintendo has developed by Genre

In [35]:
q3 = dw.query('https://data.world/mhoangvslev/steam-games-dataset',
'''
SELECT genre AS Genre, COUNT(GENRE) AS Number_of_Games_Made
FROM vgsales
WHERE developer = "Nintendo"
GROUP BY genre
ORDER BY COUNT(GENRE) ASC
'''
)

In [36]:
(print(q3.dataframe
.to_markdown(index=None)))

| Genre      |   Number_of_Games_Made |
|:-----------|-----------------------:|
| Strategy   |                      3 |
| Shooter    |                      4 |
| Adventure  |                      5 |
| Racing     |                      5 |
| Simulation |                      5 |
| Puzzle     |                      7 |
| Sports     |                      7 |
| Misc       |                      8 |
| Action     |                     12 |
| Platform   |                     16 |


In [37]:
q3_chart = (alt.Chart(q3.dataframe, title = "Games Made by Nintendo Broken Down By Genre")
.mark_bar()
.encode(
    x="Number_of_Games_Made",
    y= alt.Y('Genre', sort = 'x'),
    color = alt.Color('Number_of_Games_Made', legend = None))
.properties(
    width = 800,
    height = 400
))

q3_chart

alt.Chart(...)

In [38]:
q3_chart.save("q3_chart.png")

## Create a tabel that shows the sales of Square Enix (developer) by year, include all of the sales groups

In [39]:
q4 = dw.query('https://data.world/mhoangvslev/steam-games-dataset',
'''
SELECT year_of_release as Year,
    SUM(na_sales)*1000000 AS North_America_Sales,
    SUM(eu_sales)*1000000 AS Europe_Sales ,
    SUM(jp_sales)*1000000 AS Japan_Sales,
    SUM(other_sales)*1000000 AS Other_Sales,
    SUM(global_sales)*1000000 AS Global_Sales 
FROM vgsales
WHERE developer = "Square Enix" and year_of_release IS NOT NULL
GROUP BY year_of_release
ORDER BY year_of_release
'''
)

In [40]:
(print(q4.dataframe
.to_markdown(index=None,floatfmt='.0f')))

|   Year |   North_America_Sales |   Europe_Sales |   Japan_Sales |   Other_Sales |   Global_Sales |
|-------:|----------------------:|---------------:|--------------:|--------------:|---------------:|
|   2003 |               1680000 |         860000 |       1500000 |        130000 |        4160000 |
|   2004 |                120000 |          90000 |             0 |         30000 |         250000 |
|   2005 |               2330000 |         680000 |       1850000 |        200000 |        5060000 |
|   2006 |               3630000 |         520000 |       3730000 |       2000000 |        9880000 |
|   2007 |               2730000 |         930000 |       1680000 |        620000 |        5970000 |
|   2008 |                760000 |         700000 |       1090000 |        370000 |        2910000 |
|   2009 |               2450000 |        1410000 |       2270000 |        600000 |        6720000 |
|   2010 |               2940000 |        1590000 |       1660000 |        640000 |        

In [41]:
q4_chart_na = (alt.Chart(q4.dataframe, title = "Sales of Sqaure Enix in North America")
.mark_line(line = True)
.encode(
    x=alt.X('Year:O'),
    y=alt.Y('North_America_Sales', title=""))
.properties(
    width = 800,
    height = 400
))

q4_chart_eu = (alt.Chart(q4.dataframe, title = "Sales of Sqaure Enix Games in Europe")
.mark_line(line = True, color='blue')
.encode(
    x=alt.X('Year:O'),
    y=alt.Y('Europe_Sales', title=""))
.properties(
    width = 800,
    height = 400
))

q4_chart_jp = (alt.Chart(q4.dataframe, title = "Sales of Sqaure Enix Games in Japan")
.mark_line(line = True, color='geen')
.encode(
    x=alt.X('Year:O'),
    y=alt.Y('Japan_Sales', title=""))
.properties(
    width = 800,
    height = 400
))

q4_chart_other = (alt.Chart(q4.dataframe, title = "Sales of Sqaure Enix Games in Other Areas")
.mark_line(line = True, color='purple')
.encode(
    x=alt.X('Year:O'),
    y=alt.Y('Other_Sales', title=""))
.properties(
    width = 800,
    height = 400
))

q4_chart_global = (alt.Chart(q4.dataframe, title = "Sales of Sqaure Enix Games In Global Sales")
.mark_line(line = True, color='orange')
.encode(
    x=alt.X('Year:O'),
    y=alt.Y('Global_Sales', title=""))
.properties(
    width = 800,
    height = 400
))

In [42]:
q4_chart = alt.vconcat(q4_chart_na,q4_chart_eu,q4_chart_jp,q4_chart_other,q4_chart_global)

q4_chart

alt.VConcatChart(...)

In [43]:
q4_chart.save("q4_chart.png")